In [2]:
from music21 import *
import numpy as np
from midiutil import *
import csv

In [3]:
# song = corpus.parse('bwv66.6')
# #song.show('text')

In [7]:
paths = corpus.getComposer('bach')
song = converter.parse(paths[0])

In [38]:
for p in range(0, 10):
    song = converter.parse(paths[p])
    arr = get_array_rep(song)
    with open("bach/bw_take_3_" + str(p)+ ".csv","w", newline='') as my_csv:
        csvWriter = csv.writer(my_csv)
        csvWriter.writerows(arr)

In [31]:
arr

array([[52.  ,  1.  , 48.  ,  1.  , 45.  ,  1.  , 45.  ,  1.  ],
       [55.  ,  1.  , 48.  ,  1.  , 43.  ,  1.  , 40.  ,  1.  ],
       [57.  ,  1.  , 48.  ,  0.5 , 48.  ,  1.  , 41.  ,  1.  ],
       [55.  ,  1.  , 50.  ,  0.5 , 48.  ,  1.  , 36.  ,  0.5 ],
       [60.  ,  1.  , 52.  ,  0.5 , 48.  ,  0.5 , 38.  ,  0.5 ],
       [60.  ,  1.  , 53.  ,  0.5 , 50.  ,  0.5 , 40.  ,  0.5 ],
       [59.  ,  1.  , 55.  ,  1.  , 52.  ,  0.5 , 41.  ,  0.5 ],
       [60.  ,  1.  , 55.  ,  1.  , 50.  ,  0.5 , 43.  ,  1.  ],
       [60.  ,  1.  , 55.  ,  1.  , 50.  ,  1.  , 31.  ,  1.  ],
       [59.  ,  1.  , 55.  ,  1.  , 52.  ,  1.  , 36.  ,  1.  ],
       [57.  ,  1.  , 55.  ,  1.  , 52.  ,  1.  , 36.  ,  1.  ],
       [62.  ,  1.5 , 55.  ,  1.  , 50.  ,  1.  , 43.  ,  1.  ],
       [60.  ,  0.5 , 53.  ,  0.5 , 50.  ,  1.  , 38.  ,  0.5 ],
       [59.  ,  1.  , 52.  ,  0.5 , 45.  ,  0.5 , 40.  ,  0.5 ],
       [60.  ,  1.  , 50.  ,  0.5 , 47.  ,  0.5 , 42.  ,  0.5 ],
       [59.  ,  1.  , 52.

In [13]:
def MidiStringToInt(midstr):
    Notes = [["C"],["C#","Db"],["D"],["D#","Eb"],["E"],["F"],["F#","Gb"],["G"],["G#","Ab"],["A"],["A#","Bb"],["B"]]
    answer = 0
    i = 0
    #Note
    letter = midstr.split('-')[0].upper()
    for note in Notes:
        for form in note:
            if letter.upper() == form:
                answer = i
                break;
        i += 1
    #Octave
    answer += (int(midstr[-1]))*12
    return answer

In [35]:
def get_array_rep(song):
    partStream = song.getElementsByClass(stream.Part)
   # print(len(partStream))
    #partStream.show('text')

    #part.show('text')

    parts = []
    for p in partStream:
        part = []
        for n in p.flat.notes:
            note_with_num = MidiStringToInt(str(n.pitch.name) + "-" + str(n.pitch.octave))
            part.append([note_with_num, n.duration.quarterLength])
        parts.append(part)
    #print(parts_)
    list_len = [len(p) for p in parts]
#     final_times = [p[-1][2] for p in parts]
#     max_final_time = max(final_times)
    max_len = max(list_len)
    results_arr = None
    for part in parts:
        num_notes_missing = max_len - len(part)
        if num_notes_missing > 0:
            notes_to_add = [[0, 0] for i in range(0, num_notes_missing)]
            part = part + notes_to_add
        #print(part)
        np_part = np.array(part)
        if results_arr is None:
            results_arr = np_part
        else:
            results_arr = np.concatenate((results_arr, np_part), axis=1)
    return results_arr

In [69]:
#degrees  = [60, 62, 64, 65, 67, 69, 71, 72]  # MIDI note number
#convert array rep to midi
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 60   # In BPM
volume   = 100  # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1)  # One track
MyMIDI.addTempo(track, time, tempo)

for ch, part_ in enumerate(all_notes):
    for note in part_:
        if int(note[0]) != 0:
            MyMIDI.addNote(track, ch, int(note[0]), note[1], note[2], volume)

with open("midi-to-vector-take-2.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)